In [1]:
import os
import sys
import shutil
import numpy as np
sys.path.insert(0, '../scripts')

from eeg_cnn_lib import transfer_all_class, prepare_data_labels
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import optimizers

# The images are in a folder named 'shapes/training'
train_dir_name = '../../../data/classification/valence/train'
validation_dir_name = '../../../data/classification/valence/validation'
test_dir_name = '../../../data/classification/valence/test'

# All images are 32 pixels
img_size = (32,32)
img_size_rgba = (32,32,4)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(train_dir_name))

Using TensorFlow backend.


In [2]:
import sys
import keras
print('Keras version:',keras.__version__)

from keras import backend as K

Keras version: 2.3.1


In [3]:
transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
transfer_all_class(test_dir_name,train_dir_name,1.0,classes)

In [4]:
X, Y = prepare_data_labels(classes, train_dir_name)
X=np.asarray(X)
Y=np.asarray(Y)

In [5]:
def create_new_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides=1, padding='same', input_shape=train_generator.image_shape, activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2),strides=2))

    model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=train_generator.image_shape, activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2),strides=2))

    model.add(Flatten())

    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.2))
    
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    opt = optimizers.Adam(lr=0.001)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [6]:
kf = KFold(n_splits=5, shuffle=True)
accuracies = []

In [7]:
for train_index, val_index in kf.split(X, Y):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
    for index in range(len(X_val)):
        class_label=''
        if(Y_val[index]==0):
            class_label=classes[0]
        else:
            class_label=classes[1]  
        #Then, copy the validation images to the validation folder
        shutil.move(train_dir_name+"/"+class_label+'/'+X_val[index], 
                    validation_dir_name+"/"+class_label+'/'+X_val[index])
        
    batch_size = 256
    epoch=200
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_directory(
        train_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')
    
    model = create_new_model()
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
        
    loss, acc = model.evaluate_generator(validation_generator)
    accuracies.append(acc)
    print(acc)

Found 6839 images belonging to 2 classes.
Found 1710 images belonging to 2 classes.
Epoch 1/200
27/27 [==============================] - 4s 148ms/step - loss: 0.7140 - accuracy: 0.5118
Epoch 2/200
27/27 [==============================] - 4s 135ms/step - loss: 0.6922 - accuracy: 0.5265
Epoch 3/200
27/27 [==============================] - 4s 142ms/step - loss: 0.6914 - accuracy: 0.5255
Epoch 4/200
27/27 [==============================] - 4s 158ms/step - loss: 0.6908 - accuracy: 0.5249
Epoch 5/200
27/27 [==============================] - 4s 151ms/step - loss: 0.6896 - accuracy: 0.5334
Epoch 6/200
27/27 [==============================] - 4s 151ms/step - loss: 0.6867 - accuracy: 0.5431
Epoch 7/200
27/27 [==============================] - 4s 152ms/step - loss: 0.6849 - accuracy: 0.5562
Epoch 8/200
27/27 [==============================] - 4s 154ms/step - loss: 0.6846 - accuracy: 0.5524
Epoch 9/200
27/27 [==============================] - 4s 144ms/step - loss: 0.6796 - accuracy: 0.5670
Epoch 1

27/27 [==============================] - 7s 247ms/step - loss: 0.2719 - accuracy: 0.8801
Epoch 81/200
27/27 [==============================] - 7s 253ms/step - loss: 0.2759 - accuracy: 0.8801
Epoch 82/200
27/27 [==============================] - 7s 244ms/step - loss: 0.2736 - accuracy: 0.8773
Epoch 83/200
27/27 [==============================] - 7s 251ms/step - loss: 0.2694 - accuracy: 0.8783
Epoch 84/200
27/27 [==============================] - 7s 242ms/step - loss: 0.2691 - accuracy: 0.8827
Epoch 85/200
27/27 [==============================] - 7s 248ms/step - loss: 0.2592 - accuracy: 0.8876
Epoch 86/200
27/27 [==============================] - 7s 250ms/step - loss: 0.2513 - accuracy: 0.8936
Epoch 87/200
27/27 [==============================] - 7s 248ms/step - loss: 0.2499 - accuracy: 0.8896
Epoch 88/200
27/27 [==============================] - 7s 244ms/step - loss: 0.2440 - accuracy: 0.8957
Epoch 89/200
27/27 [==============================] - 7s 247ms/step - loss: 0.2495 - accuracy: 

27/27 [==============================] - 7s 277ms/step - loss: 0.1182 - accuracy: 0.9532
Epoch 160/200
27/27 [==============================] - 7s 267ms/step - loss: 0.1087 - accuracy: 0.9585
Epoch 161/200
27/27 [==============================] - 7s 255ms/step - loss: 0.1260 - accuracy: 0.9516
Epoch 162/200
27/27 [==============================] - 7s 263ms/step - loss: 0.1198 - accuracy: 0.9496
Epoch 163/200
27/27 [==============================] - 7s 274ms/step - loss: 0.1117 - accuracy: 0.9572
Epoch 164/200
27/27 [==============================] - 7s 264ms/step - loss: 0.1144 - accuracy: 0.9532
Epoch 165/200
27/27 [==============================] - 7s 266ms/step - loss: 0.1088 - accuracy: 0.9561
Epoch 166/200
27/27 [==============================] - 7s 261ms/step - loss: 0.1175 - accuracy: 0.9541
Epoch 167/200
27/27 [==============================] - 7s 265ms/step - loss: 0.1154 - accuracy: 0.9529
Epoch 168/200
27/27 [==============================] - 7s 270ms/step - loss: 0.1104 - a

27/27 [==============================] - 7s 256ms/step - loss: 0.4608 - accuracy: 0.7656
Epoch 39/200
27/27 [==============================] - 7s 257ms/step - loss: 0.4543 - accuracy: 0.7745
Epoch 40/200
27/27 [==============================] - 8s 285ms/step - loss: 0.4484 - accuracy: 0.7745
Epoch 41/200
27/27 [==============================] - 8s 290ms/step - loss: 0.4353 - accuracy: 0.7855
Epoch 42/200
27/27 [==============================] - 7s 278ms/step - loss: 0.4340 - accuracy: 0.7878
Epoch 43/200
27/27 [==============================] - 7s 265ms/step - loss: 0.4216 - accuracy: 0.7985
Epoch 44/200
27/27 [==============================] - 8s 291ms/step - loss: 0.4192 - accuracy: 0.7921
Epoch 45/200
27/27 [==============================] - 8s 294ms/step - loss: 0.4032 - accuracy: 0.8066
Epoch 46/200
27/27 [==============================] - 8s 287ms/step - loss: 0.4008 - accuracy: 0.8042
Epoch 47/200
27/27 [==============================] - 8s 289ms/step - loss: 0.3918 - accuracy: 

27/27 [==============================] - 8s 302ms/step - loss: 0.1436 - accuracy: 0.9395
Epoch 119/200
27/27 [==============================] - 8s 281ms/step - loss: 0.1334 - accuracy: 0.9482
Epoch 120/200
27/27 [==============================] - 7s 273ms/step - loss: 0.1397 - accuracy: 0.9449
Epoch 121/200
27/27 [==============================] - 8s 301ms/step - loss: 0.1352 - accuracy: 0.9456
Epoch 122/200
27/27 [==============================] - 8s 294ms/step - loss: 0.1311 - accuracy: 0.9488
Epoch 123/200
27/27 [==============================] - 7s 271ms/step - loss: 0.1356 - accuracy: 0.9453
Epoch 124/200
27/27 [==============================] - 7s 268ms/step - loss: 0.1271 - accuracy: 0.9477
Epoch 125/200
27/27 [==============================] - 7s 270ms/step - loss: 0.1384 - accuracy: 0.9440
Epoch 126/200
27/27 [==============================] - 8s 305ms/step - loss: 0.1402 - accuracy: 0.9443
Epoch 127/200
27/27 [==============================] - 8s 305ms/step - loss: 0.1369 - a

...

27/27 [==============================] - 8s 310ms/step - loss: 0.5604 - accuracy: 0.6994
Epoch 35/200
27/27 [==============================] - 7s 264ms/step - loss: 0.5569 - accuracy: 0.6957
Epoch 36/200
27/27 [==============================] - 7s 264ms/step - loss: 0.5552 - accuracy: 0.7035
Epoch 37/200
27/27 [==============================] - 7s 262ms/step - loss: 0.5472 - accuracy: 0.7089
Epoch 38/200
27/27 [==============================] - 8s 283ms/step - loss: 0.5358 - accuracy: 0.7166
Epoch 39/200
27/27 [==============================] - 9s 317ms/step - loss: 0.5288 - accuracy: 0.7147
Epoch 40/200
27/27 [==============================] - 8s 298ms/step - loss: 0.5222 - accuracy: 0.7245
Epoch 41/200
27/27 [==============================] - 8s 300ms/step - loss: 0.5149 - accuracy: 0.7399
Epoch 42/200
27/27 [==============================] - 8s 311ms/step - loss: 0.5080 - accuracy: 0.7410
Epoch 43/200
27/27 [==============================] - 9s 319ms/step - loss: 0.5089 - accuracy: 

27/27 [==============================] - 8s 295ms/step - loss: 0.2150 - accuracy: 0.9035
Epoch 115/200
27/27 [==============================] - 9s 317ms/step - loss: 0.1993 - accuracy: 0.9112
Epoch 116/200
27/27 [==============================] - 9s 316ms/step - loss: 0.1955 - accuracy: 0.9178
Epoch 117/200
27/27 [==============================] - 8s 304ms/step - loss: 0.1986 - accuracy: 0.9165
Epoch 118/200
27/27 [==============================] - 9s 323ms/step - loss: 0.1975 - accuracy: 0.9139
Epoch 119/200
27/27 [==============================] - 9s 325ms/step - loss: 0.1925 - accuracy: 0.9231
Epoch 120/200
27/27 [==============================] - 9s 319ms/step - loss: 0.2058 - accuracy: 0.9110
Epoch 121/200
27/27 [==============================] - 8s 314ms/step - loss: 0.1973 - accuracy: 0.9120
Epoch 122/200
27/27 [==============================] - 9s 321ms/step - loss: 0.2008 - accuracy: 0.9149
Epoch 123/200
27/27 [==============================] - 8s 313ms/step - loss: 0.1871 - a

27/27 [==============================] - 7s 260ms/step - loss: 0.1166 - accuracy: 0.9507
Epoch 194/200
27/27 [==============================] - 7s 274ms/step - loss: 0.1187 - accuracy: 0.9529
Epoch 195/200
27/27 [==============================] - 8s 286ms/step - loss: 0.1084 - accuracy: 0.9569
Epoch 196/200
27/27 [==============================] - 7s 267ms/step - loss: 0.1060 - accuracy: 0.9580
Epoch 197/200
27/27 [==============================] - 7s 257ms/step - loss: 0.1114 - accuracy: 0.9542
Epoch 198/200
27/27 [==============================] - 7s 261ms/step - loss: 0.1171 - accuracy: 0.9526
Epoch 199/200
27/27 [==============================] - 7s 262ms/step - loss: 0.1091 - accuracy: 0.9534
Epoch 200/200
27/27 [==============================] - 7s 260ms/step - loss: 0.1029 - accuracy: 0.9585
0.7596490979194641
Found 6840 images belonging to 2 classes.
Found 1709 images belonging to 2 classes.
Epoch 1/200
27/27 [==============================] - 7s 274ms/step - loss: 0.7173 - acc

27/27 [==============================] - 7s 258ms/step - loss: 0.3307 - accuracy: 0.8436
Epoch 73/200
27/27 [==============================] - 7s 251ms/step - loss: 0.3212 - accuracy: 0.8539
Epoch 74/200
27/27 [==============================] - 7s 256ms/step - loss: 0.3308 - accuracy: 0.8494
Epoch 75/200
27/27 [==============================] - 7s 252ms/step - loss: 0.3116 - accuracy: 0.8614
Epoch 76/200
27/27 [==============================] - 7s 254ms/step - loss: 0.3065 - accuracy: 0.8596
Epoch 77/200
27/27 [==============================] - 7s 258ms/step - loss: 0.3039 - accuracy: 0.8589
Epoch 78/200
27/27 [==============================] - 7s 253ms/step - loss: 0.2992 - accuracy: 0.8658
Epoch 79/200
27/27 [==============================] - 7s 258ms/step - loss: 0.3006 - accuracy: 0.8637
Epoch 80/200
27/27 [==============================] - 7s 258ms/step - loss: 0.2919 - accuracy: 0.8610
Epoch 81/200
27/27 [==============================] - 7s 254ms/step - loss: 0.2852 - accuracy: 

Epoch 152/200
27/27 [==============================] - 7s 256ms/step - loss: 0.1258 - accuracy: 0.9493
Epoch 153/200
27/27 [==============================] - 7s 249ms/step - loss: 0.1220 - accuracy: 0.9503
Epoch 154/200
27/27 [==============================] - 7s 254ms/step - loss: 0.1248 - accuracy: 0.9510
Epoch 155/200
27/27 [==============================] - 7s 253ms/step - loss: 0.1241 - accuracy: 0.9506
Epoch 156/200
27/27 [==============================] - 7s 258ms/step - loss: 0.1235 - accuracy: 0.9523
Epoch 157/200
27/27 [==============================] - 7s 250ms/step - loss: 0.1354 - accuracy: 0.9480
Epoch 158/200
27/27 [==============================] - 7s 256ms/step - loss: 0.1327 - accuracy: 0.9469
Epoch 159/200
27/27 [==============================] - 7s 255ms/step - loss: 0.1179 - accuracy: 0.9532
Epoch 160/200
27/27 [==============================] - 7s 251ms/step - loss: 0.1215 - accuracy: 0.9520
Epoch 161/200
27/27 [==============================] - 7s 255ms/step - lo

In [8]:
np.average(accuracies)

0.7676910877227783